# Candi Borobudur Chabot With RAG 

In [72]:
import torch
torch.cuda.empty_cache()

## Load Data
For reducing hallucatination from the output of the LLM, the LLM will be reinforced with external knowledge from :
- **Scrapped borobudur information** (Paper,sites,..)

#### Scrapped Information

In [73]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("Data/passage.txt",encoding="utf8")
text_data = loader.load()

**Example**

In [74]:
text_data[0].page_content

'Candi Borobudur (bahasa Jawa: ꦕꦟ꧀ꦝꦶꦧꦫꦧꦸꦝꦸꦂ,translit. Candhi Båråbudhur) adalah sebuah candi Buddha yang terletak di Borobudur, Magelang, Jawa Tengah, Indonesia. Candi ini terletak kurang lebih 100 km di sebelah barat daya Semarang, 86 km di sebelah barat Surakarta, dan 40 km di sebelah barat laut Yogyakarta. Candi dengan banyak stupa ini didirikan oleh para penganut agama Buddha Mahayana sekitar tahun 800-an Masehi pada masa pemerintahan wangsa Syailendra. Borobudur adalah candi atau kuil Buddha terbesar di dunia sekaligus salah satu monumen Buddha terbesar di dunia.\n\n\n\n\nPada tahun 1907, di bawah pimpinan van Erp dimulai pemugaran Candi Borobudur untuk pertama kalinya, dengan dana f 34.600 atas usul van Erp. van Erp berhasil menyelesaikan tiga lapis chatra di atas yasthi stupa induk, dengan dokumen foto yang dikerjakan Kasiyan Cepaas pada tahun 1911. setelah van Erp, pemugaran berikutnya pada tahun 1925-1926 dipimpin oleh Theodoor van der Groen van Rijk. Fokus pemugaran kedua ini

## Process Data

The additonal data would be processed so that searches across the data can run more easily

#### Split Text

In [75]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_split = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap = 200)

passage_split = text_split.split_documents(text_data)

print('=================================================')
print(f'Passage Data Split Amount : {len(passage_split)}')
print(f'Sample                    : {passage_split[0].page_content}')

Passage Data Split Amount : 81
Sample                    : Candi Borobudur (bahasa Jawa: ꦕꦟ꧀ꦝꦶꦧꦫꦧꦸꦝꦸꦂ,translit. Candhi Båråbudhur) adalah sebuah candi Buddha yang terletak di Borobudur, Magelang, Jawa Tengah, Indonesia. Candi ini terletak kurang lebih 100 km di sebelah barat daya Semarang, 86 km di sebelah barat Surakarta, dan 40 km di sebelah barat laut Yogyakarta. Candi dengan banyak stupa ini didirikan oleh para penganut agama Buddha Mahayana sekitar tahun 800-an Masehi pada masa pemerintahan wangsa Syailendra. Borobudur adalah candi atau kuil Buddha terbesar di dunia sekaligus salah satu monumen Buddha terbesar di dunia.


#### Embedding
Instantiate Embedding model that responsible for transforming data into numerical representation in a continuous vector space

In [76]:
import os
os.environ["PYDANTIC_SKIP_VALIDATING_CORE_SCHEMAS"] = "True"

In [77]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# Load an Indonesian sentence embedding model from Hugging Face
indosentencebert_embeddings = HuggingFaceEmbeddings(model_name="firqaaa/indo-sentence-bert-base")

In [78]:
# Let's say you want the embedding for a specific sentence
sentence = "Candi Borobudur adalah situs warisan dunia di Indonesia."

# Get the embedding vector (a list of floats)
embedding_vector = indosentencebert_embeddings.embed_query(sentence)

# Print the embedding vector
print(embedding_vector)

# Optional: check its dimension
print(f"Embedding length: {len(embedding_vector)}")

[-0.1491147130727768, -0.2169771045446396, -1.0847731828689575, -0.4401751458644867, -0.28182557225227356, -0.9626078009605408, 1.6356391906738281, -0.30013784766197205, -0.4362701177597046, 0.3844544589519501, -1.112185001373291, 0.6294882297515869, 0.022857386618852615, 0.11551296710968018, -0.5756826996803284, -1.2429490089416504, -0.5645748376846313, -0.3133658468723297, 0.2947027385234833, -0.24240368604660034, -1.1904921531677246, 1.91094172000885, -0.5004600286483765, 0.35473352670669556, -0.2983098328113556, -0.3620080351829529, -0.44770553708076477, 0.5120301842689514, -0.46747294068336487, 0.37058258056640625, -0.8789358735084534, 1.829017996788025, 0.263571560382843, 0.25506410002708435, 0.02394457347691059, -0.8327537178993225, 0.1694180965423584, -0.5216978788375854, 0.18812832236289978, 1.3918185234069824, -0.2455044835805893, 0.0623847134411335, 0.7079011797904968, 0.15163441002368927, -1.2753254175186157, -0.19298353791236877, 0.873906672000885, 0.5393505096435547, 0.70

## Create RAG Components

#### Retriever

In [79]:
from langchain.vectorstores import FAISS

db_passages = FAISS.from_documents(passage_split, embedding=indosentencebert_embeddings)

In [80]:
retriever_passages = db_passages.as_retriever(
    search_type="similarity", search_kwargs={"k": 5, "include_metadata": True}
)

In [81]:
retriever_passages.invoke("Apa itu candi borobudur?")

[Document(id='b44e8d57-4317-401a-80ce-8c9bb0e2be5b', metadata={'source': 'Data/passage.txt'}, page_content='Candi Borobudur (bahasa Jawa: ꦕꦟ꧀ꦝꦶꦧꦫꦧꦸꦝꦸꦂ,translit. Candhi Båråbudhur) adalah sebuah candi Buddha yang terletak di Borobudur, Magelang, Jawa Tengah, Indonesia. Candi ini terletak kurang lebih 100 km di sebelah barat daya Semarang, 86 km di sebelah barat Surakarta, dan 40 km di sebelah barat laut Yogyakarta. Candi dengan banyak stupa ini didirikan oleh para penganut agama Buddha Mahayana sekitar tahun 800-an Masehi pada masa pemerintahan wangsa Syailendra. Borobudur adalah candi atau kuil Buddha terbesar di dunia sekaligus salah satu monumen Buddha terbesar di dunia.'),
 Document(id='c3e50e5b-bba7-4b19-9877-8451855015d8', metadata={'source': 'Data/passage.txt'}, page_content='Candi Borobudur (bahasa Jawa: ꦕꦟ꧀ꦝꦶꦧꦫꦧꦸꦝꦸꦂ,translit. Candhi Båråbudhur) adalah sebuah candi Buddha yang terletak di Borobudur, Magelang, Jawa Tengah, Indonesia. Candi ini terletak kurang lebih 100 km di sebe

#### LLM
Instantiate LLM for question-answering system

In [ ]:
import os
from langchain_groq import ChatGroq

llm = ChatGroq(temperature=0.4, model_name="llama-3.1-8b-instant",max_tokens=256,api_key="")

## Build RAG App

#### Create History

In [100]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompts import ChatPromptTemplate

# Define a prompt to turn a question with chat history context
# into a clear, standalone question without using the history.
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

# Create a prompt template that combines system instructions, chat history, and user input.
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),  # Placeholder for chat history
        ("human", "{input}"),  # Placeholder for the user's latest question
    ]
)

# Create a retriever that understands the chat history using the LLM, compression retriever, and the prompt template.
history_aware_retriever = create_history_aware_retriever(
    llm, retriever_passages, contextualize_q_prompt
)

#### Create Retrieval Chain

In [101]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain


# Define a system prompt for the assistant, named "Bori", who specializes in answering questions
system_prompt = (
    "Kamu adalah asisten bernama Bori, singkatan dari *Borobudur Story*, yang bertugas menjawab pertanyaan seputar Candi Borobudur. "
    "Gunakan potongan konteks yang diberikan untuk membantu menjawab pertanyaan. "
    "Jika kamu tidak mengetahui jawabannya, katakan bahwa kamu tidak tahu. "
    "Jawaban harus singkat, maksimal tiga kalimat, dan jangan menjawab pertanyaan yang tidak berkaitan dengan Candi Borobudur. "
    "Jawablah dengan nada yang ramah dan bersahabat, seperti sedang mengobrol santai namun informatif.\n\n"
    "{context}"
)

# Create a chat prompt template that combines the system instructions, chat history, and user input.
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),              # System instructions (how Bori should behave)
        MessagesPlaceholder("chat_history"),    # Placeholder for the chat history
        ("human", "{input}"),                   # Placeholder for the user's latest question
    ]
)

# Create a chain for answering questions based on the context retrieved.
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# Combine the history-aware retriever and the question-answer chain to form a retrieval-augmented generation (RAG) chain.
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

## Testing

In [102]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

In [103]:
question = "Siapa ketua prodi informatika sekarang?"
ai_msg = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg["answer"]),
    ]
)
ai_msg['answer']

'Maaf, saya tidak tahu jawabannya. Saya adalah asisten Bori yang berfokus pada sejarah dan informasi tentang Candi Borobudur, jadi saya tidak memiliki informasi tentang ketua prodi informatika.'

#### Save Requirements

In [125]:
!pip freeze > requirements.txt

## Evaluation
Evaluation would be test on several LLM such as Lllama70b & GoogleGemini

In [105]:
def calculate_bleu_score(standard_answer,generated_answer):
  import evaluate

  # Load the BLEU evaluation metric
  bleu = evaluate.load("bleu")

  predictions = [generated_answer]
  references = [standard_answer]

  # Compute the BLEU score
  results = bleu.compute(predictions=predictions,references=references)

  return results

In [106]:
def calculate_rouge_score(standard_answer,generated_answer):
  import evaluate

  # Load the ROUGE evaluation metric
  rouge = evaluate.load('rouge')

  predictions = [generated_answer]
  references = [standard_answer]

  # Compute the ROUGE score
  results = rouge.compute(predictions=predictions,references=references)

  return results

#### Context Chain

In [107]:
from langchain import HuggingFacePipeline, PromptTemplate, LLMChain
import pandas as pd

prompt_context = PromptTemplate.from_template( """
          System: Anda adalah chatbot interaktif yang asik untuk menjawab pertanyaan. Gunakan potongan konteks yang diambil berikut ini untuk menjawab pertanyaan tidak apa untuk bilang tidak tahu.
          {Context}
          User: {Question}
          Assistant:
          """
)


standard_qa_pairs = pd.read_csv('Data/Data Konteks Chatbot Candi Borobudur - All.csv')

standard_qa_pairs.head()

,Konteks,Pertanyaan,Jawaban,Topik
0,"Candi Borobudur (bahasa Jawa: ꦕꦟ꧀ꦝꦶꦧꦫꦧꦸꦝꦸꦂ,tra...",Di mana letak geografis Candi Borobudur?,"Candi Borobudur terletak di Borobudur, Magelan...",Budaya
1,"Candi Borobudur (bahasa Jawa: ꦕꦟ꧀ꦝꦶꦧꦫꦧꦸꦝꦸꦂ,tra...",Berapa jarak antara Candi Borobudur dan Semarang?,Candi Borobudur terletak kurang lebih 100 km d...,Budaya
2,"Candi Borobudur (bahasa Jawa: ꦕꦟ꧀ꦝꦶꦧꦫꦧꦸꦝꦸꦂ,tra...",Siapa yang mendirikan Candi Borobudur?,Candi Borobudur didirikan oleh para penganut a...,Budaya
3,"Candi Borobudur (bahasa Jawa: ꦕꦟ꧀ꦝꦶꦧꦫꦧꦸꦝꦸꦂ,tra...",Pada masa pemerintahan mana Candi Borobudur di...,Candi Borobudur didirikan pada masa pemerintah...,Budaya
4,"Candi Borobudur (bahasa Jawa: ꦕꦟ꧀ꦝꦶꦧꦫꦧꦸꦝꦸꦂ,tra...",Apa status Candi Borobudur di dunia?,Borobudur adalah candi atau kuil Buddha terbes...,Budaya


In [120]:
import logging
logging.getLogger('absl').setLevel(logging.ERROR)

In [121]:
from time import time
import warnings
import pandas as pd
from sentence_transformers import SentenceTransformer, util

warnings.filterwarnings("ignore")

# Load IndoBERT model once
indobert_model = SentenceTransformer("firqaaa/indo-sentence-bert-base")

# Global variables
total_inference_time = 0
total_pairs = 0
df = pd.DataFrame()

# Function to process each row
def process_qa_pair(row):
    global total_inference_time, total_pairs

    question = row["Pertanyaan"]
    standard_answer = row["Jawaban"]
    context = row["Konteks"]
    topic = row["Topik"]

    # Inference with LLM
    start = time()
    generated_answer = llm_chain_context({"Context": context, "Question": question})["text"].replace("\n", "")
    end = time()
    inference_time = end - start

    total_inference_time += inference_time
    total_pairs += 1

    if len(generated_answer.strip()) == 0:
        generated_answer = "Undefined"

    # Calculate metrics
    bleu_score = calculate_bleu_score(standard_answer, generated_answer)
    rouge_score = calculate_rouge_score(standard_answer, generated_answer)

    # === IndoBERT Similarity ===
    embeddings = indobert_model.encode([standard_answer, generated_answer])
    indobert_similarity = float(util.cos_sim(embeddings[0], embeddings[1]))

    # Combine results
    merged_data = {
        "Question": question,
        "Generated Answer": generated_answer,
        "Standard Answer": standard_answer,
        "Topic": topic,
        "BLEU Score": bleu_score,
        "ROUGE Score": rouge_score,
        "IndoBERT Similarity": indobert_similarity,
        "Inference Time": inference_time,
    }

    # Print detail
    print(f"==== Question {total_pairs} ====")
    print(f"Context: {context}")
    print(f"Question: {question}")
    print(f"Generated Answer: {generated_answer}")
    print(f"Standard Answer: {standard_answer}")
    print(f"Inference Time: {round(inference_time, 3)} seconds")
    print(f"BLEU Score: {bleu_score}")
    print(f"ROUGE Score: {rouge_score}")
    print(f"IndoBERT Similarity: {round(indobert_similarity, 3)}")
    print("="*50 + "\n")

    return pd.Series(merged_data)

In [122]:
llm_chain_context = LLMChain(prompt=prompt_context ,llm=llm)

total_inference_time = 0  # Variable to store total inference time
total_pairs = 0  # Variable to store total number of question-answer pairs

# Apply the function to each row of the DataFrame
df_llama_3_8b = standard_qa_pairs.apply(process_qa_pair, axis=1)

# Print total inference time and total pairs if desired
print(f"Total Inference Time: {total_inference_time}")
print(f"Total Number of Pairs: {total_pairs}")

# Calculate average inference time
average_inference_time = total_inference_time / total_pairs
print(f"Average Inference Time: {round(average_inference_time,3)} seconds")

df_llama_3_8b.to_csv('Result/hasil_uji_keseluruhan.csv')

==== Question 1 ====
Context: Candi Borobudur (bahasa Jawa: ꦕꦟ꧀ꦝꦶꦧꦫꦧꦸꦝꦸꦂ,translit. Candhi Båråbudhur) adalah sebuah candi Buddha yang terletak di Borobudur, Magelang, Jawa Tengah, Indonesia. Candi ini terletak kurang lebih 100 km di sebelah barat daya Semarang, 86 km di sebelah barat Surakarta, dan 40 km di sebelah barat laut Yogyakarta. Candi dengan banyak stupa ini didirikan oleh para penganut agama Buddha Mahayana sekitar tahun 800-an Masehi pada masa pemerintahan wangsa Syailendra. Borobudur adalah candi atau kuil Buddha terbesar di dunia sekaligus salah satu monumen Buddha terbesar di dunia.
Question: Di mana letak geografis Candi Borobudur?
Generated Answer: Candi Borobudur terletak di Borobudur, Magelang, Jawa Tengah, Indonesia. Lebih tepatnya, candi ini berada sekitar 100 km di sebelah barat daya Semarang, 86 km di sebelah barat Surakarta, dan 40 km di sebelah barat laut Yogyakarta.
Standard Answer: Candi Borobudur terletak di Borobudur, Magelang, Jawa Tengah, Indonesia.
Infere

In [124]:
for topic_name, topic_df in standard_qa_pairs.groupby("Topik"):
    print(f"=== Processing Topik: {topic_name} ===")
    
    # Reset hitungan waktu dan jumlah pasang QA per topik
    total_inference_time = 0
    total_pairs = 0
    
    result_df = topic_df.apply(process_qa_pair, axis=1)

    # Simpan ke CSV
    filename = f"hasil_uji_{topic_name.lower().replace(' ', '_')}.csv"
    result_df.to_csv(filename, index=False)
    
    # Cetak ringkasan
    if total_pairs > 0:
        avg_time = total_inference_time / total_pairs
        print(f"Rata-rata Waktu Inference untuk topik '{topic_name}': {round(avg_time, 3)} detik\n")
    else:
        print(f"Tidak ada pasangan pertanyaan-jawaban untuk topik '{topic_name}'\n")

=== Processing Topik: Arsitektur ===
==== Question 1 ====
Context: "Platform melingkar di Candi Borobudur, yang terletak di tingkat 7, 8, 9, dan 10, berfungsi sebagai representasi simbolis dari kepala, langit, atau elemen tak berbentuk. Platform ini adalah bagian dari Arupadhatu, tingkat tertinggi kuil, dan menandakan perjalanan spiritual menuju pencerahan. Bentuk melingkar dari platform mewakili konsep mandala, diagram kosmik yang melambangkan alam semesta dan keterkaitan segala sesuatu. Mereka juga mewakili tingkat kesadaran dan pencapaian spiritual yang lebih tinggi, saat perjalanan berlangsung menuju tujuan akhir pencerahan. Platform melingkar, bersama dengan elemen arsitektur candi lainnya, menciptakan ruang suci untuk kontemplasi dan meditasi, memfasilitasi pertumbuhan spiritual individu. Secara keseluruhan, tujuan dari platform melingkar di Candi Borobudur adalah untuk memberikan representasi simbolis dan pengalaman dari perjalanan spiritual menuju pencerahan dan keterkaitan sem

In [115]:
import os
import pandas as pd
import ast
import re

# Fungsi untuk membersihkan string 'np.float64(...)' -> angka
def clean_np_float(string):
    return re.sub(r'np\.float64\((.*?)\)', r'\1', string)

# Fungsi untuk memproses satu file dan return dict rata-rata skor
def process_csv(file_path):
    df = pd.read_csv(file_path)

    # Clean BLEU and ROUGE
    df["BLEU Score"] = df["BLEU Score"].apply(lambda x: ast.literal_eval(clean_np_float(x)) if isinstance(x, str) else x)
    df["ROUGE Score"] = df["ROUGE Score"].apply(lambda x: ast.literal_eval(clean_np_float(x)) if isinstance(x, str) else x)

    # Extract scores
    df["BLEU"] = df["BLEU Score"].apply(lambda x: x.get("bleu", 0.0))
    df["ROUGE-1"] = df["ROUGE Score"].apply(lambda x: float(x.get("rouge1", 0.0)))
    df["ROUGE-2"] = df["ROUGE Score"].apply(lambda x: float(x.get("rouge2", 0.0)))
    df["ROUGE-L"] = df["ROUGE Score"].apply(lambda x: float(x.get("rougeL", 0.0)))
    df["IndoBERT Similarity"] = df["IndoBERT Similarity"].astype(float)

    # Return ringkasan hasil
    return {
        "Topik": os.path.basename(file_path).replace("hasil_uji_", "").replace(".csv", "").capitalize(),
        "BLEU": df["BLEU"].mean(),
        "ROUGE-1": df["ROUGE-1"].mean(),
        "ROUGE-2": df["ROUGE-2"].mean(),
        "ROUGE-L": df["ROUGE-L"].mean(),
        "IndoBERT Similarity": df["IndoBERT Similarity"].mean()
    }


In [117]:
# Folder hasil
folder_path = "./Result"  # Ubah jika lokasinya berbeda


# Ambil semua file CSV dalam folder
csv_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".csv")]

# Proses semua file
summary_data = [process_csv(f) for f in csv_files]

# Buat DataFrame ringkasan dan simpan
summary_df = pd.DataFrame(summary_data)
summary_df

,Topik,BLEU,ROUGE-1,ROUGE-2,ROUGE-L,IndoBERT Similarity
0,Arsitektur,0.082135,0.375874,0.191764,0.306712,0.753369
1,Budaya,0.372667,0.599684,0.510612,0.554048,0.860124
2,Keseluruhan,0.255972,0.512875,0.385085,0.456742,0.831763
3,Pemugaran,0.391240,0.606128,0.513287,0.561989,0.873809
4,Sejarah,0.202110,0.472304,0.335620,0.423435,0.801871


In [119]:
summary_df.to_csv("Result/ringkasan_evaluasi.csv", index=False)

print("✅ Rekap selesai dan disimpan ke ringkasan_evaluasi.csv")

✅ Rekap selesai dan disimpan ke ringkasan_evaluasi.csv
